# Introduction

**References**  
This notebook was built using the following amazing resources created by :
- **Martin Gorner:** [Getting Started: TPUs + Cassava Leaf Disease](https://www.kaggle.com/jessemostipak/getting-started-tpus-cassava-leaf-disease)

The Output of the notbook can be used in the inference notebook proviede earlier:

https://www.kaggle.com/lucamtb/brain-tumor-very-basice-inference

# Set up environment

In [ ]:
import math, re, os, random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

# Set up variables


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = KaggleDatasets().get_gcs_path('tfflair-records')
BATCH_SIZE = 24 * strategy.num_replicas_in_sync
IMAGE_SIZE = [256, 256] # was 512
EPOCHS = 5
folds = 3



In [ ]:
GCS_PATH

In [ ]:
df_gs = pd.DataFrame(tf.io.gfile.glob(GCS_PATH + '/*.tfrec'),columns = ['gs'])

df_gs

In [ ]:
from sklearn.model_selection import GroupKFold


gkf  = GroupKFold(n_splits = folds)
df_gs['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df_gs, groups = df_gs.gs.tolist())):

    df_gs.loc[val_idx, 'fold'] = fold

df_gs



# Load the data
If you've primarily worked with notebooks in Learn, you've maybe noticed that data import and formatting is taken care of for you. But because we're working with competition data we'll have to handle this part of the pipeline ourselves.   

The data we're working with have been formatted into `TFRecords`, which are a format for storing a sequence of binary records. `TFRecords` work _really_ well with TPUs, and allow us to send a small number of large files across the TPU for processing.   

If you'd like to learn more about `TFRecords` and maybe even try creating them yourself, check out this **[TFRecords Basics notebook](https://www.kaggle.com/ryanholbrook/tfrecords-basics)** and **[corresponding video](https://youtu.be/KgjaC9VeOi8)** from Kaggle Data Scientist Ryan Holbrook.  

Because our data consists of `training` and `test` images only, we're going to split our `training` data into `training` and `validation` data using the `train_test_split()` function. 

## Decode the data
In the code chunk below we'll set up a series of functions that allow us to convert our images into tensors so that we can utilize them in our model. We'll also normalize our data. Our images are using a "Red, Blue, Green (RBG)" scale that has a range of [0, 255], and by normalizing it we'll set each pixel's value to a number in the range of [0, 1]. 

In [ ]:
[*IMAGE_SIZE]

In [ ]:
def decode_image(image):
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image,[*IMAGE_SIZE])
    #image = tf.reshape(image, [*IMAGE_SIZE])
    return image

In [ ]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
         "label": tf.io.FixedLenSequenceFeature([], tf.int64,allow_missing=True)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    label = tf.cast(example['label'], tf.int32)
    #label = tf.reshape(label,(1,num_classes))
  
    return image, label

We'll use the following function to load our dataset. One of the advantages of a TPU is that we can run multiple files across the TPU at once, and this accounts for the speed advantages of using a TPU. To capitalize on that, we want to make sure that we're using data as soon as it streams in, rather than creating a data streaming bottleneck.

In [ ]:
def filter_fn(image, label):
    
    im_size = tf.math.reduce_mean(image)
    
    return tf.math.greater(im_size, 0.025)

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    dataset = dataset.filter(filter_fn)
    return dataset

## A note on using train_test_split()
While I used `train_test_split()` to create both a `training` and `validation` dataset, consider exploring **[cross validation instead](https://www.kaggle.com/dansbecker/cross-validation)**.

In [ ]:
#TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')

In [ ]:
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH + '/*.tfrec'),
    test_size=0.2, random_state=15 ## was 0.35
)

#TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')

In [ ]:
dataset = load_dataset(TRAINING_FILENAMES, labeled=True)

In [ ]:
for i in dataset.take(2):
    
    print(i[0].shape)

In [ ]:
dataset = tf.data.TFRecordDataset(VALID_FILENAMES, num_parallel_reads=AUTOTUNE)

In [ ]:
dataset = dataset.with_options(tf.data.Options())

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

labeled = True

tfrecord_format = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "label": tf.io.FixedLenSequenceFeature([], tf.int64,allow_missing=True)
} if labeled else {
    "image": tf.io.FixedLenFeature([], tf.string),
    "image_name": tf.io.FixedLenFeature([], tf.string)
    }

j = 0
for i in dataset:
    image = tf.io.parse_single_example(i,tfrecord_format)
    if j == 1:
        break
    else:
        j = j +1
        #print(tf.reshape(tf.reduce_max(tf.one_hot(image['image'], num_classes, dtype=tf.int32), axis=0),(1,19)))



In [ ]:
image = decode_image(image['image'])

In [ ]:
tf.math.reduce_mean(image)

In [ ]:
dataset.take(1)

In [ ]:
image.shape

In [ ]:
plt.imshow(image)

## Adding in augmentations 


In [ ]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    ##image = tf.image.random_crop(image, [int(IMAGE_SIZE[0]), int(IMAGE_SIZE[0]), 3])
    image = tf.image.rot90(image, k= random.randint(1, 4))
    #image = dropout(image)
    #label = tf.one_hot(label, num_classes, dtype=tf.int32)
    #image = tf.image.draw_bounding_boxes(image, [64, 1, 32] , colors)
    #image = tfa.image.rotate(image, tf.constant(np.pi/8))
    #image = tf.image.random_brightness(image, 0.2)
    #image = tf.image.random_hue(image, 0.2)
    #image = tf.image.random_saturation(image, 5, 10)
    return image, label

## Define data loading methods
The following functions will be used to load our `training`, `validation`, and `test` datasets, as well as print out the number of images in each dataset.

In [ ]:
def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)  
    #dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)  
    dataset = dataset.repeat()
    dataset = dataset.shuffle(30233)
    #dataset = dataset.unbatch()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered) 
    dataset = dataset.batch(BATCH_SIZE)
    #dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALID_FILENAMES)
#NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

print('Dataset: {} training images, {} validation images '.format(
    NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES))

In [ ]:
NUM_VALIDATION_IMAGES

In [ ]:
dataset = load_dataset(TRAINING_FILENAMES, labeled=True) 
dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE) 

In [ ]:
for i in dataset.take(3):
    print((i[0].shape))

You can also modify the above code to look at your `validation` and `test` data, like this:

In [ ]:
# load our validation dataset for EDA
validation_dataset = get_validation_dataset()
validation_dataset = validation_dataset.unbatch().batch(20)
valid_batch = iter(validation_dataset)

## Building our model
In order to ensure that our model is trained on the TPU, we build it using `with strategy.scope()`.    

This model was built using transfer learning, meaning that we have a _pre-trained model_ (ResNet50) as our base model and then the customizable model built using `tf.keras.Sequential`. If you're new to transfer learning I recommend setting `base_model.trainable` to **False**, but _do_ encourage you to change which base model you're using (more options are available in the **[`tf.keras.applications` Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications)** documentation) as well iterate on the custom model. 

Note that we're using `sparse_categorical_crossentropy` as our loss function, because we did _not_ one-hot encode our labels.

# Train the model
As our model is training you'll see a printout for each epoch, and can also monitor TPU usage by clicking on the TPU metrics in the toolbar at the top right of your notebook.

In [ ]:
# load data
train_dataset = get_training_dataset()
valid_dataset = get_validation_dataset()

# Model


In [ ]:
!pip install -U efficientnet

In [ ]:
import efficientnet.keras as eff
from keras.layers import Dropout


In [ ]:
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALID_STEPS = NUM_VALIDATION_IMAGES // BATCH_SIZE

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers:#[-18:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
             layer.trainable = True

In [ ]:
num_folds = [i for i in range(folds)]

lr_rates = [3e-5]

epsilons = [1e-4]

for fold in num_folds:


    for lr in lr_rates:

        for e in epsilons:

            my_callbacks = [
                #tf.keras.callbacks.EarlyStopping(patience=5,verbose=1,monitor='val_binary_accuracy'),
                tf.keras.callbacks.ModelCheckpoint(filepath=f'Brain_flair_model_effect_{lr}_{e}.h5',verbose=1,monitor='val_binary_accuracy',save_best_only=True),
                #tf.keras.callbacks.LearningRateScheduler(lr_schedul,verbose=1)
                tf.keras.callbacks.ReduceLROnPlateau(monitor='val_binary_accuracy',factor=0.1,patience=3,min_lr=1e-30,mode='min',verbose=1,)

            ]

            with strategy.scope():       
                #img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
                img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.densenet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])
                #img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.resnet.preprocess_input, input_shape=[*IMAGE_SIZE, 3])

                #base_model = eff.EfficientNetB0(weights='noisy-student', include_top=False) #imagenet noisy-student
                base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False)
                #base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
                base_model.trainable = False

                model = tf.keras.Sequential([
                    tf.keras.layers.BatchNormalization(renorm=True),
                    img_adjust_layer,
                    base_model,
                    tf.keras.layers.GlobalAveragePooling2D(),  ## Average
                    tf.keras.layers.Dense(512, activation='relu', 
                    bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)),
                    Dropout(0.5),
                    #tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)), # L2 normalize embeddings
                    #tf.keras.layers.Dense(1024, activation='relu', 
                    #bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)),
                    #Dropout(0.5),

                    #tf.keras.layers.BatchNormalization(renorm=True),
                    tf.keras.layers.Dense(1, activation='sigmoid')  
                ])

                model.compile(
                    optimizer=tf.keras.optimizers.Adam(learning_rate=lr,epsilon=e),
                    #loss='binary_crossentropy',#loss_func,  
                    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                    metrics=tf.keras.metrics.BinaryAccuracy()
                )


                unfreeze_model(model)

                epochs = EPOCHS  # @param {type: "slider", min:8, max:50}

                history = model.fit(train_dataset, 
                                    steps_per_epoch=STEPS_PER_EPOCH, 
                                    epochs=epochs,
                                    #validation_split=0.1,
                                    validation_data=valid_dataset,
                                    #validation_steps=VALID_STEPS,
                                   callbacks=my_callbacks)

                history_frame = pd.DataFrame(history.history)
                history_frame.loc[:, ['loss', 'val_loss']].plot()
                history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();



